# Projekt Apache Spark

# Wprowadzenie

Wykorzystując ten notatnik jako szablon zrealizuj projekt Apache Spark zgodnie z przydzielonym zestawem. 

Kilka uwag:

* Nie modyfikuj ani nie usuwaj paragrafów *markdown* w tym notatniku, chyba że wynika to jednoznacznie z instrukcji. 
* Istniejące paragrafy zawierające *kod* uzupełnij w razie potrzeby zgodnie z instrukcjami
    - nie usuwaj ich
    - nie usuwaj zawartych w nich instrukcji oraz kodu
    - nie modyfikuj ich, jeśli instrukcje jawnie tego nie nakazują
* Możesz dodawać nowe paragrafy zarówno zawierające kod jak i komentarze dotyczące tego kodu (markdown)

# Zestaw 6 – fifa-players

## Misja główna

### Cel przetwarzania

Dla następujących kategorii: 
- narodowość (opartej na `nationality_name`)
- klub (opartej na `club_name`)
- liga (opartej na `league_name`)

należy wyznaczyć trzy wartości (nazwę narodowości/klubu/ligi) z piłkarzami o największych średnich zarobkach. 

W analizach nie uwzględniamy piłkarzy grających w ligach, w których liczba grających zespołów mających co najmniej 11 piłkarzy jest mniejsza niż 10. 

Wynik przetwarzania powinien zawierać następujące atrybuty: 
- `category` – nazwa kategorii (`nationality`, `club`, `league`)
- `name` – nazwa narodowości/klubu/ligi
- `sum_value_eur` – sumaryczna wartość piłkarzy
- `avg_wage_eur` – średnie zarobki piłkarzy 
- `avg_age` – średni wiek piłkarzy w lidze obliczony na podstawie wartości kolumny age 
(nie korzystamy z `dob`)
- `count_players` – liczba zawodników 
- `player_positions` – lista skrótów nazw pozycji, na których grają piłkarze 

Sugerowany schemat wyniku:
```
root
 |-- category: string (nullable = false)
 |-- name: string (nullable = true)
 |-- sum_value_eur: double (nullable = true)
 |-- avg_wage_eur: double (nullable = true)
 |-- avg_age: double (nullable = true)
 |-- count_players: long (nullable = false)
 |-- player_positions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 ```

Uwagi:
- lista skrótów nazw pozycji, na których grają piłkarze nie może zawierać duplikatów 
- oryginalne wartości kolumny player_positions mogą zawierać wiele skrótów nazw pozycji, należy je traktować jako zbiór
- wartości liczbowe mają być zaokrąglone do jedności

# Działania wstępne 

Uruchom poniższy paragraf, aby utworzyć obiekty kontekstu Sparka. Jeśli jest taka potrzeba dostosuj te polecenia. Pamiętaj o potrzebnych bibliotekach.

In [1]:
from pyspark.sql import SparkSession
import os

# Spark session & context
spark = SparkSession.builder.appName("FifaPlayers").getOrCreate()
sc = spark.sparkContext  # Tworzymy alias dla SparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/29 15:19:23 INFO SparkEnv: Registering MapOutputTracker
24/12/29 15:19:23 INFO SparkEnv: Registering BlockManagerMaster
24/12/29 15:19:23 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/12/29 15:19:23 INFO SparkEnv: Registering OutputCommitCoordinator


W poniższym paragrafie uzupełnij polecenia definiujące poszczególne zmienne. 

Pamiętaj abyś:

* w późniejszym kodzie, dla wszystkich cześci projektu, korzystał z tych zdefiniowanych zmiennych. Wykorzystuj je analogicznie jak parametry
* przed ostateczną rejestracją projektu usunął ich wartości, tak aby nie pozostawiać w notatniku niczego co mogłoby identyfikować Ciebie jako jego autora

In [2]:
# pełna ścieżka do katalogu w zasobniku zawierającego podkatalogi `datasource1` i `datasource4` 
# z danymi źródłowymi
input_dir = "gs://pdb-24-mn/project1/input"

Nie modyfikuj poniższych paragrafów. Wykonaj je i używaj zdefniowanych poniżej zmiennych jak parametrów Twojego programu.

In [3]:
# NIE ZMIENIAĆ
# ścieżki dla danych źródłowych 
datasource1_dir = input_dir + "/datasource1"
datasource4_dir = input_dir + "/datasource4"

# nazwy i ścieżki dla wyników dla misji głównej 
# część 1 (Spark Core - RDD) 
rdd_result_dir = "/tmp/output1"

# część 2 (Spark SQL - DataFrame)
df_result_table = "output2"

# część 3 (Pandas API on Spark)
ps_result_file = "/tmp/output3.json"

In [4]:
# NIE ZMIENIAĆ
import os
def remove_file(file):
    if os.path.exists(file):
        os.remove(file)

remove_file("metric_functions.py")
remove_file("tools_functions.py")

In [5]:
# NIE ZMIENIAĆ
import requests
r = requests.get("https://jankiewicz.pl/bigdata/metric_functions.py", allow_redirects=True)
open('metric_functions.py', 'wb').write(r.content)
r = requests.get("https://jankiewicz.pl/bigdata/tools_functions.py", allow_redirects=True)
open('tools_functions.py', 'wb').write(r.content)

3322

In [6]:
# NIE ZMIENIAĆ
%run metric_functions.py
%run tools_functions.py

Poniższe paragrafy mają na celu usunąć ewentualne pozostałości poprzednich uruchomień tego lub innych notatników

In [7]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 1 (Spark Core - RDD) 
delete_dir(spark, rdd_result_dir)

Deleted /tmp/output1
Successfully deleted file in HDFS: /tmp/output1


In [8]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 2 (Spark SQL - DataFrame) 
drop_table(spark, df_result_table)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


The physical location of the table output2 is: hdfs://pbd-cluster-m/user/hive/warehouse/output2


rm: `hdfs://pbd-cluster-m/user/hive/warehouse/output2': No such file or directory


Error deleting file hdfs://pbd-cluster-m/user/hive/warehouse/output2: Command '['hadoop', 'fs', '-rm', '-r', 'hdfs://pbd-cluster-m/user/hive/warehouse/output2']' returned non-zero exit status 1.


In [9]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 3 (Pandas API on Spark) 
remove_file(ps_result_file)

In [10]:
# NIE ZMIENIAĆ
spark

***Uwaga!***

Uruchom poniższy paragraf i sprawdź czy adres, pod którym dostępny *Apache Spark Application UI* jest poprawny wywołując następny testowy paragraf. 

W razie potrzeby określ samodzielnie poprawny adres, pod którym dostępny *Apache Spark Application UI*

In [11]:
# adres URL, pod którym dostępny Apache Spark Application UI (REST API)
# 
spark_ui_address = extract_host_and_port(spark, "http://localhost:4041")
spark_ui_address

'http://pbd-cluster-m.europe-west4-c.c.big-data-2024-09-mn.internal:37777'

In [12]:
# testowy paragraf
test_metrics = get_current_metrics(spark_ui_address)
test_metrics

{'numTasks': 0,
 'numActiveTasks': 0,
 'numCompleteTasks': 0,
 'numFailedTasks': 0,
 'numKilledTasks': 0,
 'numCompletedIndices': 0,
 'executorDeserializeTime': 0,
 'executorDeserializeCpuTime': 0,
 'executorRunTime': 0,
 'executorCpuTime': 0,
 'resultSize': 0,
 'jvmGcTime': 0,
 'resultSerializationTime': 0,
 'memoryBytesSpilled': 0,
 'diskBytesSpilled': 0,
 'peakExecutionMemory': 0,
 'inputBytes': 0,
 'inputRecords': 0,
 'outputBytes': 0,
 'outputRecords': 0,
 'shuffleRemoteBlocksFetched': 0,
 'shuffleLocalBlocksFetched': 0,
 'shuffleFetchWaitTime': 0,
 'shuffleRemoteBytesRead': 0,
 'shuffleRemoteBytesReadToDisk': 0,
 'shuffleLocalBytesRead': 0,
 'shuffleReadBytes': 0,
 'shuffleReadRecords': 0,
 'shuffleWriteBytes': 0,
 'shuffleWriteTime': 0,
 'shuffleWriteRecords': 0}

# Część 1 - Spark Core (RDD)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [13]:
# NIE ZMIENIAĆ
before_rdd_metrics = get_current_metrics(spark_ui_address)
before_rdd_metrics

{'numTasks': 0,
 'numActiveTasks': 0,
 'numCompleteTasks': 0,
 'numFailedTasks': 0,
 'numKilledTasks': 0,
 'numCompletedIndices': 0,
 'executorDeserializeTime': 0,
 'executorDeserializeCpuTime': 0,
 'executorRunTime': 0,
 'executorCpuTime': 0,
 'resultSize': 0,
 'jvmGcTime': 0,
 'resultSerializationTime': 0,
 'memoryBytesSpilled': 0,
 'diskBytesSpilled': 0,
 'peakExecutionMemory': 0,
 'inputBytes': 0,
 'inputRecords': 0,
 'outputBytes': 0,
 'outputRecords': 0,
 'shuffleRemoteBlocksFetched': 0,
 'shuffleLocalBlocksFetched': 0,
 'shuffleFetchWaitTime': 0,
 'shuffleRemoteBytesRead': 0,
 'shuffleRemoteBytesReadToDisk': 0,
 'shuffleLocalBytesRead': 0,
 'shuffleReadBytes': 0,
 'shuffleReadRecords': 0,
 'shuffleWriteBytes': 0,
 'shuffleWriteTime': 0,
 'shuffleWriteRecords': 0}

W poniższych paragrafach wprowadź **rozwiązanie** *misji głównej* oparte na *RDD API*. 

Pamiętaj o wydajności Twojego przetwarzania, *RDD API* tego wymaga. 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [14]:
# Funkcja, która zamienia wartość na liczbę, a w przypadku błędnych danych zwraca 0
def safe_number(value):
    try:
        return float(value) if value is not None else 0
    except (ValueError, TypeError):
        return 0  # Zwracamy 0 w przypadku błędu konwersji

# Funkcja, która zamienia wartość na listę, a w przypadku błędnych danych zwraca pustą listę
def safe_list(value):
    return value if isinstance(value, list) else []

# Funkcja zaokrąglająca wartości liczbowe do jednostek
def round_value(value):
    return round(value)  # Zaokrąglanie do najbliższej liczby całkowitej

In [15]:
# Załaduj dane z datasource1
rdd1 = sc.textFile(datasource1_dir).repartition(os.cpu_count())
datasource1_rdd = rdd1.map(lambda line: line.split(";")) \
    .map(lambda fields: (
        fields[0],  # player_id (kolumna 0)
        fields[16],  # league_id (klucz)
        fields[18],  # club_name
        fields[17],  # club_team_id
        fields[10],  # value_eur
        fields[11],  # wage_eur
        fields[12],  # age
        fields[7].split(", "),  # player_positions (split po przecinku)
        fields[25]  # nationality
    ))

# Załaduj dane z datasource4
rdd4 = sc.textFile(datasource4_dir).repartition(os.cpu_count())
datasource4_rdd = rdd4.map(lambda line: line.split(";")) \
    .map(lambda fields: (
        fields[0],  # league_id (klucz)
        fields[1]   # league_name
    ))

In [16]:
# Połączenie RDD (left join)
merged_rdd = datasource1_rdd.map(lambda row: (row[1], row)) \
    .leftOuterJoin(datasource4_rdd) \
    .map(lambda row: (
        row[1][0][0],  # player_id
        row[1][0][2],  # club_name
        row[1][0][3],  # club_team_id
        row[1][0][4],  # value_eur
        row[1][0][5],  # wage_eur 
        row[1][0][6],  # age
        row[1][0][7],  # player_positions
        row[1][0][8],  # nationality
        row[1][1] if row[1][1] else "Unknown"  # league_name lub "Unknown"
    ))


In [17]:
# Tworzymy RDD z (league_name, club_name)
club_player_rdd = merged_rdd.map(lambda row: (row[8], row[1]))  # (league_name, club_name)

# Zliczanie liczby piłkarzy w każdym klubie
players_count_by_club_rdd = club_player_rdd.map(lambda x: ((x[0], x[1]), 1)) \
    .reduceByKey(lambda a, b: a + b)  # Zliczanie liczby piłkarzy per (league_name, club_id)

# Filtrujemy kluby, które mają co najmniej 11 piłkarzy
valid_clubs_rdd = players_count_by_club_rdd.filter(lambda x: x[1] >= 11)

# Grupowanie wyników po lidze, liczenie liczby klubów w danej lidze
clubs_in_leagues_rdd = valid_clubs_rdd.map(lambda x: (x[0][0], 1))  # Zwracamy tylko league_name, liczba 1 dla każdego klubu
valid_clubs_in_leagues_rdd = clubs_in_leagues_rdd.reduceByKey(lambda x, y: x + y)  # Zliczanie liczby klubów w lidze

# Filtrujemy ligi, w których jest przynajmniej 10 klubów z co najmniej 11 piłkarzami
valid_leagues_rdd = valid_clubs_in_leagues_rdd.filter(lambda x: x[1] >= 10)

# # Debugging: wyświetlanie wyników
# print("Ligi, które spełniają warunek:")
# for league, count in valid_leagues_rdd.collect():
#     print(f"Liga {league} ma {count} klubów z co najmniej 11 piłkarzami")


In [18]:
# Filtrujemy merged_rdd, aby zawierało tylko zawodników, którzy grają w ligach spełniających warunki
filtered_merged_rdd = merged_rdd.map(lambda row: (row[8], row)) \
    .join(valid_leagues_rdd.map(lambda x: (x[0], None)))  # Join z valid_leagues_rdd, żeby tylko te ligi, które spełniają warunki

# Zwracamy tylko same rekordy zawodników
filtered_merged_rdd = filtered_merged_rdd.map(lambda x: x[1][0])

In [19]:
# Zamiana wartości null na domyślne (0 dla liczb, pusta lista dla player_positions)
processed_rdd = filtered_merged_rdd.map(lambda row: (
    row[0],  # player_id
    row[1],  # club_name
    row[2],  # club_team_id
    safe_number(row[3]),  # value_eur (zamiana null na 0)
    safe_number(row[4]),  # wage_eur (zamiana null na 0)
    safe_number(row[5]),  # age (zamiana null na 0)
    safe_list(row[6]),    # player_positions (zamiana null na pustą listę)
    row[7],  # nationality
    row[8] if row[8] else "Unknown"  # league_name
))

In [20]:
# Funkcja pomocnicza, aby wybrać top N z RDD
def get_top_n(rdd, n):
    return rdd.sortBy(lambda x: x[3], ascending=False).zipWithIndex().filter(lambda x: x[1] < n).map(lambda x: x[0])

In [21]:
# Załaduj dane z processed_rdd (top_nationalities_rdd)
nationality_stats = processed_rdd.map(lambda row: (
    row[7],  # nationality
    (row[4], row[3], row[5], 1)  # wage_eur, value_eur, age, count
)).reduceByKey(lambda a, b: (
    a[0] + b[0],  # Suma wage_eur
    a[1] + b[1],  # Suma value_eur
    a[2] + b[2],  # Suma wieku
    a[3] + b[3],  # Liczba graczy
))

# Obliczanie średnich zarobków, wieku i sumy wartości
nationality_avg_stats = nationality_stats.map(lambda x: (
    'nationality',                   # category
    x[0],                            # name (nationality)
    round_value(x[1][1]),            # sum_value_eur (zaokrąglone)
    round_value(x[1][0] / x[1][3]),  # avg_wage_eur (zaokrąglone)
    round_value(x[1][2] / x[1][3]),  # avg_age (zaokrąglone)
    x[1][3],                         # count_players
))

# Sortowanie po sumie wartości (value_eur)
top_nationalities_rdd = get_top_n(nationality_avg_stats, 3)

# Tworzenie RDD z pozycji narodowości
nationality_positions_rdd = processed_rdd.flatMap(lambda row: [(row[7], position) for position in row[6]])  # (nationality, position)

# Usuwanie duplikatów pozycji (distinct) dla każdej narodowości
nationality_positions_rdd_distinct = nationality_positions_rdd.distinct()  # Usuwamy duplikaty pozycji

# Grupowanie po narodowościach i zbieranie unikalnych pozycji
nationality_positions_grouped = nationality_positions_rdd_distinct.groupByKey().mapValues(list)

# Łączenie top_nationalities_rdd z nationality_positions_grouped po narodowości (left outer join)
final_nationalities_rdd = top_nationalities_rdd.map(lambda x: (
    x[1],  # 'category', 'name', 'value', 'wage', 'age', 'count'
    (x[2],  # sum_value_eur
     x[3],  # avg_wage_eur
     x[4],  # avg_age
     x[5],  # count_players
    )  
)).leftOuterJoin(nationality_positions_grouped) \
  .map(lambda x: (
      'nationality',    # category
      x[0],             # name
      x[1][0][0],       # sum_value_eur
      x[1][0][1],       # avg_wage_eur
      x[1][0][2],       # avg_age
      x[1][0][3],       # count_players
      x[1][1] if x[1][1] else []  # positions, default to empty list if no positions
))

final_nationalities_rdd = final_nationalities_rdd.sortBy(lambda x: x[2], ascending=False)

In [22]:
# Grupowanie po ligach
league_stats = processed_rdd.map(lambda row: (
    row[8],  # league_name
    (row[4], row[3], row[5], 1)  # wage_eur, value_eur, age, count
)).reduceByKey(lambda a, b: (
    a[0] + b[0],  # Suma wage_eur
    a[1] + b[1],  # Suma value_eur
    a[2] + b[2],  # Suma wieku
    a[3] + b[3]   # Liczba graczy
))

# Obliczanie średnich zarobków, wieku i sumy wartości oraz zaokrąglanie dla lig
league_avg_stats = league_stats.map(lambda x: (
    'league',                        # category
    x[0],                            # name
    round_value(x[1][1]),            # sum_value_eur (zaokrąglone)
    round_value(x[1][0] / x[1][3]),  # avg_wage_eur (zaokrąglone)
    round_value(x[1][2] / x[1][3]),  # avg_age (zaokrąglone)
    x[1][3]                          # count_players
))

# Sortowanie po sumie wartości (value_eur)
top_leagues_rdd = get_top_n(league_avg_stats, 3)

# Tworzenie RDD z pozycji lig
league_positions_rdd = processed_rdd.flatMap(lambda row: [(row[8], position) for position in row[6]])  # (league_name, position)

# Usuwanie duplikatów pozycji (distinct) dla każdej ligi
league_positions_rdd_distinct = league_positions_rdd.distinct()  # Usuwamy duplikaty pozycji

# Grupowanie po ligach i zbieranie unikalnych pozycji
league_positions_grouped = league_positions_rdd_distinct.groupByKey().mapValues(list)

# Łączenie top_leagues_rdd z league_positions_grouped po nazwie ligi (left outer join)
final_leagues_rdd = top_leagues_rdd.map(lambda x: (
    x[1],  # 'category', 'name', 'value', 'wage', 'age', 'count'
    (x[2],  # sum_value_eur
     x[3],  # avg_wage_eur
     x[4],  # avg_age
     x[5],  # count_players
    )  
)).leftOuterJoin(league_positions_grouped) \
  .map(lambda x: (
      'league',        # category
      x[0],            # name
      x[1][0][0],      # sum_value_eur
      x[1][0][1],      # avg_wage_eur
      x[1][0][2],      # avg_age
      x[1][0][3],      # count_players
      x[1][1] if x[1][1] else []  # positions, default to empty list if no positions
))

# Sortowanie wynikowego RDD po sum_value_eur malejąco
final_leagues_rdd = final_leagues_rdd.sortBy(lambda x: x[2], ascending=False)

In [23]:
# Załaduj dane z processed_rdd (top_clubs_rdd)
club_stats = processed_rdd.map(lambda row: (
    row[1],  # club_name
    (row[4], row[3], row[5], 1)  # wage_eur, value_eur, age, count
)).reduceByKey(lambda a, b: (
    a[0] + b[0],  # Suma wage_eur
    a[1] + b[1],  # Suma value_eur
    a[2] + b[2],  # Suma wieku
    a[3] + b[3],  # Liczba graczy
))

# Obliczanie średnich zarobków, wieku i sumy wartości
club_avg_stats = club_stats.map(lambda x: (
    'club',                         # category
    x[0],                            # name (club_name)
    round_value(x[1][1]),            # sum_value_eur (zaokrąglone)
    round_value(x[1][0] / x[1][3]),  # avg_wage_eur (zaokrąglone)
    round_value(x[1][2] / x[1][3]),  # avg_age (zaokrąglone)
    x[1][3],                         # count_players
))

# Sortowanie po sumie wartości (value_eur)
top_clubs_rdd = get_top_n(club_avg_stats, 3)

# Tworzenie RDD z pozycji klubów
club_positions_rdd = processed_rdd.flatMap(lambda row: [(row[1], position) for position in row[6]])  # (club_name, position)

# Usuwanie duplikatów pozycji (distinct) dla każdego klubu
club_positions_rdd_distinct = club_positions_rdd.distinct()  # Usuwamy duplikaty pozycji

# Grupowanie po klubach i zbieranie unikalnych pozycji
club_positions_grouped = club_positions_rdd_distinct.groupByKey().mapValues(list)

# Łączenie top_clubs_rdd z club_positions_grouped po nazwie klubu (left outer join)
final_clubs_rdd = top_clubs_rdd.map(lambda x: (
    x[1],  # 'category', 'name', 'value', 'wage', 'age', 'count'
    (x[2],  # sum_value_eur
     x[3],  # avg_wage_eur
     x[4],  # avg_age
     x[5],  # count_players
    )  
)).leftOuterJoin(club_positions_grouped) \
  .map(lambda x: (
      'club',          # category
      x[0],            # name
      x[1][0][0],      # sum_value_eur
      x[1][0][1],      # avg_wage_eur
      x[1][0][2],      # avg_age
      x[1][0][3],      # count_players
      x[1][1] if x[1][1] else []  # positions, default to empty list if no positions
))

final_clubs_rdd = final_clubs_rdd.sortBy(lambda x: x[2], ascending=False)

In [24]:
# Łączenie wyników z każdej kategorii (top 3 narodowości, lig i klubów)
final_top_9_rdd = final_clubs_rdd.union(final_leagues_rdd).union(final_nationalities_rdd)

# Wyświetlenie zawartości RDD (całość top 9)
final_top_9_list = final_top_9_rdd.collect()  # Pobierz wszystkie dane do lokalnej listy
print("\nFinal Top 9:")
for item in final_top_9_list:
    print(item)

# Zapisanie wyniku
final_top_9_rdd.saveAsPickleFile(rdd_result_dir)


Final Top 9:
('club', 'Manchester City', 1281910000, 101385, 25, 39, ['ST', 'CF', 'RB', 'CAM', 'RM', 'GK', 'LB', 'RWB', 'CB', 'CM', 'LM', 'LW', 'CDM', 'RW'])
('club', 'FC Barcelona', 1131875000, 91571, 24, 49, ['RWB', 'CM', 'CB', 'GK', 'ST', 'CF', 'CDM', 'LB', 'RB', 'RW', 'LW', 'LM', 'RM', 'CAM'])
('club', 'Real Madrid', 1050650000, 118934, 24, 38, ['CM', 'CB', 'RW', 'LW', 'RB', 'CDM', 'CAM', 'RM', 'LB', 'GK', 'ST', 'CF'])
('league', 'Premier League', 11049435000, 21002, 24, 2295, ['RWB', 'GK', 'RW', 'RB', 'CDM', 'LM', 'LW', 'CM', 'CB', 'LWB', 'LB', 'ST', 'CF', 'RM', 'CAM'])
('league', 'Serie A', 7729900000, 19340, 26, 1544, ['GK', 'CDM', 'RW', 'LB', 'LWB', 'CAM', 'RM', 'RB', 'RWB', 'LW', 'LM', 'ST', 'CF', 'CB', 'CM'])
('league', 'La Liga', 7612290000, 21643, 25, 1143, ['RW', 'CAM', 'RM', 'LWB', 'ST', 'CF', 'CM', 'CB', 'LB', 'RWB', 'GK', 'RB', 'LW', 'LM', 'CDM'])
('nationality', 'Brazil', 7025990000, 13337, 27, 2440, ['LM', 'LW', 'ST', 'CF', 'GK', 'LB', 'RM', 'CAM', 'RB', 'CM', 'CB', 

In [25]:
# testowy paragraf
test_metrics = get_current_metrics(spark_ui_address)
# Wybierz tylko te trzy interesujące parametry
selected_metrics = {key: value for key, value in test_metrics.items() if key in ['shuffleReadBytes', 'shuffleWriteBytes', 'inputBytes']}
# Wypisz wynik
print(selected_metrics)

{'inputBytes': 34499267, 'shuffleReadBytes': 33597143, 'shuffleWriteBytes': 22903187}


Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [26]:
# NIE ZMIENIAĆ
after_rdd_metrics = get_current_metrics(spark_ui_address)
#print(after_rdd_metrics)

# Część 2 - Spark SQL (DataFrame)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [27]:
# NIE ZMIENIAĆ
before_df_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź **rozwiązanie** *misji głównej* swojego projektu oparte o *DataFrame API*. 

Pamiętaj o wydajności Twojego przetwarzania, *DataFrame API* nie jest w stanie wszystkiego "naprawić". 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [28]:
from pyspark.sql import functions as F

In [29]:
# Załaduj dane z datasource1
df1 = spark.read.option("delimiter", ";").csv(datasource1_dir, header=False)
df1 = df1.select(
    df1._c0.alias("player_id"),   # player_id (kolumna 0)
    df1._c16.alias("league_id"),   # league_id (klucz)
    df1._c18.alias("club_name"),   # club_name
    df1._c17.alias("club_team_id"), # club_team_id
    df1._c10.alias("value_eur"),  # value_eur
    df1._c11.alias("wage_eur"),   # wage_eur
    df1._c12.alias("age"),        # age
    F.split(df1._c7, ", ").alias("player_positions"), # player_positions (split po przecinku)
    df1._c25.alias("nationality") # nationality
)

# Załaduj dane z datasource4
df4 = spark.read.option("delimiter", ";").csv(datasource4_dir, header=False)
df4 = df4.select(
    df4._c0.alias("league_id"),   # league_id (klucz)
    df4._c1.alias("league_name")  # league_name
)

In [30]:
# Połączenie DataFrame (left join) na podstawie 'league_id'
merged_df = df1.join(df4, on="league_id", how="left") \
    .select(
        df1.player_id,           # player_id
        df1.club_name,           # club_name
        df1.club_team_id,        # club_team_id
        df1.value_eur,           # value_eur
        df1.wage_eur,            # wage_eur
        df1.age,                 # age
        df1.player_positions,    # player_positions
        df1.nationality,         # nationality
        df4.league_name           # league_name (z df4, aby uniknąć niejednoznaczności)
    )

# Zamieniamy NULL na 'Unknown' w kolumnie 'league_name'
merged_df = merged_df.withColumn("league_name", F.coalesce(df4.league_name, F.lit("Unknown")))

In [31]:
# Grupowanie danych po lidze i klubie, zliczanie liczby unikalnych piłkarzy w klubie
club_player_df = merged_df.select(
    "league_name", 
    "club_name", 
    "player_id"
).distinct()  # Upewniamy się, że każdy piłkarz jest liczone tylko raz

# Liczenie liczby piłkarzy w klubie
players_count_by_club_df = club_player_df.groupBy("league_name", "club_name") \
    .agg(F.count("player_id").alias("players_count"))

# Filtrujemy kluby, które mają co najmniej 11 piłkarzy
valid_clubs_df = players_count_by_club_df.filter(F.col("players_count") >= 11)

# Grupowanie wyników po lidze, zliczanie liczby klubów w danej lidze
clubs_in_leagues_df = valid_clubs_df.groupBy("league_name") \
    .agg(F.count("club_name").alias("clubs_count"))

# Filtrujemy ligi, w których jest przynajmniej 10 klubów z co najmniej 11 piłkarzami
valid_leagues_df = clubs_in_leagues_df.filter(F.col("clubs_count") >= 10)

# Zmiana nazwy kolumny "league_name" na "valid_league_name"
valid_leagues_df = valid_leagues_df.withColumnRenamed("league_name", "valid_league_name")


In [32]:
# Wykonanie inner join pomiędzy merged_df a valid_leagues_df na odpowiednich kolumnach
processed_df = merged_df.join(valid_leagues_df, merged_df["league_name"] == valid_leagues_df["valid_league_name"], how="inner") \
    .select(
        merged_df["player_id"],        # player_id
        merged_df["club_name"],        # club_name
        merged_df["club_team_id"],     # club_team_id
        merged_df["value_eur"],        # value_eur
        merged_df["wage_eur"],         # wage_eur
        merged_df["age"],              # age
        merged_df["player_positions"], # player_positions
        merged_df["nationality"],      # nationality
        valid_leagues_df["valid_league_name"].alias("league_name") # alias dla kolumny league_name
    )

In [33]:
def calculate_top3_with_positions(processed_df, group_column, category_name):
    # Obliczanie sumy dla wszystkich wymaganych kolumn
    stats_df = processed_df.groupBy(group_column).agg(
        F.sum("value_eur").alias("sum_value_eur"),
        F.sum("age").alias("sum_age"),
        F.sum("wage_eur").alias("sum_wage_eur"),
        F.count(group_column).alias("count_players")
    )

    # Obliczanie średnich zarobków (avg_wage_eur) i średniego wieku (avg_age)
    avg_stats_df = stats_df.withColumn(
        "avg_wage_eur", F.round(F.col("sum_wage_eur") / F.col("count_players"))
    ).withColumn(
        "avg_age", F.round(F.col("sum_age") / F.col("count_players"))
    )

    # Usuwanie zbędnych kolumn sumy
    avg_stats_df = avg_stats_df.drop("sum_wage_eur").drop("sum_age")

    # Sortowanie po średnich zarobkach (avg_wage_eur) i wybieranie top 3
    top3_df = avg_stats_df.orderBy(F.col("avg_wage_eur"), ascending=False).limit(3)

    # Uzyskiwanie unikalnych pozycji dla każdej grupy
    positions_df = processed_df.select(group_column, "player_positions") \
        .withColumn("player_position", F.explode("player_positions")) \
        .dropDuplicates([group_column, "player_position"])

    # Grupowanie pozycji zawodników po grupie
    positions_grouped_df = positions_df.groupBy(group_column).agg(
        F.collect_list("player_position").alias("positions")
    )

    # Łączenie top3_df z positions_grouped_df
    final_df = top3_df.join(positions_grouped_df, on=group_column, how="left")

    # Dodanie kolumny category
    final_df = final_df.withColumn("category", F.lit(category_name))

    # Zmiana nazw kolumn
    final_df = final_df.withColumnRenamed(group_column, "name") \
        .withColumnRenamed("positions", "player_positions")

    # Zaokrąglanie kolumn sum_value_eur, avg_wage_eur i avg_age
    final_df = final_df.withColumn("sum_value_eur", F.round(F.col("sum_value_eur")))

    # Zmiana kolejności kolumn, aby pasowała do oczekiwanego wyniku
    final_df = final_df.select(
        "category", "name", "sum_value_eur", "avg_wage_eur", "avg_age", "count_players", "player_positions"
    )
    
    final_df = final_df.orderBy(F.col("avg_wage_eur"), ascending=False)

    return final_df

# Wyliczenie top3 dla każdej kategorii
final_nationalities_df = calculate_top3_with_positions(processed_df, "nationality", "nationality")
final_leagues_df = calculate_top3_with_positions(processed_df, "league_name", "league_name")
final_clubs_df = calculate_top3_with_positions(processed_df, "club_name", "club_name")

# Połączenie wyników w jeden DataFrame
final_top3_df = final_clubs_df.union(final_leagues_df).union(final_nationalities_df)

# Wyświetlenie finalnej tabeli
print("Tabela z wynikami 3xTop3 (nationality, league_name, club_name):")
final_top3_df.show(9, truncate=False)

Tabela z wynikami 3xTop3 (nationality, league_name, club_name):


+-----------+------------------+-------------+------------+-------+-------------+----------------------------------------------------------------+
|category   |name              |sum_value_eur|avg_wage_eur|avg_age|count_players|player_positions                                                |
+-----------+------------------+-------------+------------+-------+-------------+----------------------------------------------------------------+
|club_name  |Real Madrid       |1.05065E9    |118934.0    |24.0   |38           |[CDM, CAM, RM, ST, LB, RW, CB, CM, CF, RB, LW, GK]              |
|club_name  |Manchester City   |1.28191E9    |101385.0    |25.0   |39           |[CDM, LB, GK, RB, LM, CM, ST, RWB, RW, CB, LW, CAM, CF, RM]     |
|club_name  |FC Barcelona      |1.131875E9   |91571.0     |24.0   |49           |[ST, GK, CM, CDM, RB, CB, CAM, LW, RWB, RM, RW, LM, LB, CF]     |
|league_name|La Liga           |7.61229E9    |21643.0     |25.0   |1143         |[CM, CAM, ST, LW, CDM, LM, RWB, RB, L

In [34]:
# Zapis wyników jako tabela Delta Lake
final_top3_df.write.mode("overwrite").saveAsTable(df_result_table)

24/12/29 15:21:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/29 15:21:47 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [35]:
# testowy paragraf
test_metrics = get_current_metrics(spark_ui_address)
# Wybierz tylko te trzy interesujące parametry
selected_metrics = {key: value for key, value in test_metrics.items() if key in ['shuffleReadBytes', 'shuffleWriteBytes', 'inputBytes']}
# Wypisz wynik
print(selected_metrics)

{'inputBytes': 517536181, 'shuffleReadBytes': 55935799, 'shuffleWriteBytes': 45241559}


Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [36]:
# NIE ZMIENIAĆ
after_df_metrics = get_current_metrics(spark_ui_address)

# Część 3 - Pandas API on Spark

Ta część to wyzwanie. W szczególności dla osób, które nie programują na co dzień w Pythonie, lub które nie nie korzystały do tej pory z Pandas API.  

Powodzenia!

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [37]:
#NIE ZMIENIAĆ
before_ps_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź **rozwiązanie** swojego projektu oparte o *Pandas API on Spark*. 

Pamiętaj o wydajności Twojego przetwarzania, *Pandas API on Spark* nie jest w stanie wszystkiego "naprawić". 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [38]:
import pyspark.pandas as ps

# Załaduj dane z datasource1
df1 = ps.read_csv(datasource1_dir, sep=";", header=None)
df1.columns = [f"_c{i}" for i in range(len(df1.columns))]  # Tymczasowe nazwy kolumn
df1 = df1[["_c0", "_c16", "_c18", "_c17", "_c10", "_c11", "_c12", "_c7", "_c25"]]
df1.columns = [
    "player_id",         # player_id (kolumna 0)
    "league_id",         # league_id
    "club_name",         # club_name
    "club_team_id",      # club_team_id
    "value_eur",         # value_eur
    "wage_eur",          # wage_eur
    "age",               # age
    "player_positions",  # player_positions
    "nationality"        # nationality
]

# Rozdzielanie wartości w "player_positions" na listę
df1["player_positions"] = df1["player_positions"].apply(
    lambda x: x.split(", ") if isinstance(x, str) else []
)

# Załaduj dane z datasource4
df4 = ps.read_csv(datasource4_dir, sep=";", header=None)
df4.columns = [f"_c{i}" for i in range(len(df4.columns))]  # Tymczasowe nazwy kolumn
df4 = df4[["_c0", "_c1"]]
df4.columns = ["league_id", "league_name"]

/usr/lib/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
/usr/lib/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/usr/lib/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [39]:
# Połączenie DataFrame (left join) na podstawie 'league_id'
merged_df = df1.merge(df4, on="league_id", how="left")

# Wybieranie odpowiednich kolumn
merged_df = merged_df[[
    "player_id",           # player_id
    "club_name",           # club_name
    "club_team_id",        # club_team_id
    "value_eur",           # value_eur
    "wage_eur",            # wage_eur
    "age",                 # age
    "player_positions",    # player_positions
    "nationality",         # nationality
    "league_name"          # league_name (z df4)
]]


In [40]:
# Grupowanie danych po lidze i klubie, liczenie liczby piłkarzy w klubie
players_count_by_club_df = merged_df.groupby(["league_name", "club_name"]) \
    .size().reset_index(name="players_count")

# Filtrujemy kluby, które mają co najmniej 11 piłkarzy
valid_clubs_df = players_count_by_club_df[players_count_by_club_df["players_count"] >= 11]

# Grupowanie wyników po lidze, liczenie liczby klubów w danej lidze
clubs_in_leagues_df = valid_clubs_df.groupby("league_name") \
    .size().reset_index(name="clubs_count")

# Filtrujemy ligi, w których jest przynajmniej 10 klubów z co najmniej 11 piłkarzami
valid_leagues_df = clubs_in_leagues_df[clubs_in_leagues_df["clubs_count"] >= 10]

# # Debugging: wyświetlanie wyników
# print("Ligi spełniające warunek:")
# print(valid_leagues_df.head(50))


In [41]:
# Połączenie DataFrame (right join) na podstawie 'league_name' w Pandas API on Spark
processed_df = merged_df.merge(valid_leagues_df[["league_name"]], on="league_name", how="inner")

# Wybieramy odpowiednie kolumny
processed_df = processed_df[[
    "player_id",           # player_id
    "club_name",           # club_name
    "club_team_id",        # club_team_id
    "value_eur",           # value_eur
    "wage_eur",            # wage_eur
    "age",                 # age
    "player_positions",    # player_positions
    "nationality",         # nationality
    "league_name"          # league_name
]]


In [42]:
def calculate_top3_with_positions_ps(processed_df, group_column, category_name):
    
    # Obliczanie sumy dla wszystkich wymaganych kolumn, zachowanie group_column
    stats_df = processed_df.groupby(group_column).agg(
        sum_value_eur=("value_eur", "sum"),
        sum_age=("age", "sum"),
        sum_wage_eur=("wage_eur", "sum"),
        count_players=(group_column, "count")
    )

    # Obliczanie średnich zarobków (avg_wage_eur) i średniego wieku (avg_age)
    stats_df["avg_wage_eur"] = (stats_df["sum_wage_eur"] / stats_df["count_players"]).round()
    stats_df["avg_age"] = (stats_df["sum_age"] / stats_df["count_players"]).round()

    # Usuwanie zbędnych kolumn sumy
    stats_df = stats_df.drop(columns=["sum_wage_eur", "sum_age"])

    # Przywracanie kolumny group_column do stats_df przed sortowaniem
    stats_df[group_column] = stats_df.index

    # Sortowanie po średnich zarobkach (avg_wage_eur) i wybieranie top 3
    top3_df = stats_df.sort_values(by="avg_wage_eur", ascending=False).head(3)

    # Uzyskiwanie unikalnych pozycji dla każdej grupy (bez użycia set)
    positions_df = processed_df[[group_column, "player_positions"]] \
        .explode("player_positions") \
        .drop_duplicates([group_column, "player_positions"])

    # Grupowanie pozycji zawodników po grupie i tworzenie listy pozycji dla każdego rekordu
    positions_grouped_df = positions_df.groupby(group_column)["player_positions"].apply(lambda x: x.drop_duplicates().tolist()).reset_index()

    # Łączenie top3_df z positions_grouped_df
    final_df = top3_df.merge(positions_grouped_df, on=group_column, how="left")

    # Dodanie kolumny category
    final_df["category"] = category_name

    # Zmiana nazw kolumn
    final_df = final_df.rename(columns={group_column: "name", "player_positions": "player_positions"})

    # Zaokrąglanie kolumn sum_value_eur, avg_wage_eur i avg_age
    final_df["sum_value_eur"] = final_df["sum_value_eur"].round()

    # Zmiana kolejności kolumn, aby pasowała do oczekiwanego wyniku
    final_df = final_df[["category", "name", "sum_value_eur", "avg_wage_eur", "avg_age", "count_players", "player_positions"]]

    # Sortowanie wyników po średnich zarobkach
    final_df = final_df.sort_values(by="avg_wage_eur", ascending=False)

    return final_df

# Wyliczenie top3 dla każdej kategorii
final_nationalities_df = calculate_top3_with_positions_ps(processed_df, "nationality", "nationality")
final_leagues_df = calculate_top3_with_positions_ps(processed_df, "league_name", "league_name")
final_clubs_df = calculate_top3_with_positions_ps(processed_df, "club_name", "club_name")

# Połączenie wyników w jeden DataFrame przy użyciu pyspark.pandas.concat
final_top3_df = ps.concat([final_clubs_df, final_leagues_df, final_nationalities_df])

# Wyświetlenie finalnej tabeli
print("Tabela z wynikami 3xTop3 (nationality, league_name, club_name):")
print(final_top3_df)

/usr/lib/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `groupby.apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
24/12/29 15:22:09 WARN AttachDistributedSequenceExec: clean up cached RDD(384) in AttachDistributedSequenceExec(23138)
/usr/lib/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `groupby.apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
24/12/29 15:22:15 WARN AttachDistributedSequenceExec: clean up cached RDD(426) in AttachDistributedSequenceExec(24178)
/usr/lib/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `groupby.apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
24/12/29 15:22:21 WARN A

Tabela z wynikami 3xTop3 (nationality, league_name, club_name):


24/12/29 15:22:34 WARN AttachDistributedSequenceExec: clean up cached RDD(530) in AttachDistributedSequenceExec(29436)
24/12/29 15:22:38 WARN AttachDistributedSequenceExec: clean up cached RDD(544) in AttachDistributedSequenceExec(29782)


      category                name  sum_value_eur  avg_wage_eur  avg_age  count_players                                                  player_positions
0    club_name         Real Madrid   1.050650e+09      118934.0     24.0             38                [CAM, CB, CDM, CF, CM, GK, LB, LW, RB, RM, RW, ST]
1    club_name     Manchester City   1.281910e+09      101385.0     25.0             39       [CAM, CB, CDM, CF, CM, GK, LB, LM, LW, RB, RM, RW, RWB, ST]
2    club_name        FC Barcelona   1.131875e+09       91571.0     24.0             49       [CAM, CB, CDM, CF, CM, GK, LB, LM, LW, RB, RM, RW, RWB, ST]
0  league_name             La Liga   7.612290e+09       21643.0     25.0           1143  [CAM, CB, CDM, CF, CM, GK, LB, LM, LW, LWB, RB, RM, RW, RWB, ST]
1  league_name      Premier League   1.104944e+10       21002.0     24.0           2295  [CAM, CB, CDM, CF, CM, GK, LB, LM, LW, LWB, RB, RM, RW, RWB, ST]
2  league_name             Serie A   7.729900e+09       19340.0     26.0    

In [43]:
final_top3_df.to_json(ps_result_file, orient='records')

/usr/lib/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
24/12/29 15:22:56 WARN AttachDistributedSequenceExec: clean up cached RDD(659) in AttachDistributedSequenceExec(39337)
24/12/29 15:23:00 WARN AttachDistributedSequenceExec: clean up cached RDD(673) in AttachDistributedSequenceExec(39683)


In [44]:
# testowy paragraf
test_metrics = get_current_metrics(spark_ui_address)
# Wybierz tylko te trzy interesujące parametry
selected_metrics = {key: value for key, value in test_metrics.items() if key in ['shuffleReadBytes', 'shuffleWriteBytes', 'inputBytes']}
# Wypisz wynik
print(selected_metrics)

{'inputBytes': 1277186242, 'shuffleReadBytes': 78581281, 'shuffleWriteBytes': 67886491}


Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [45]:
#NIE ZMIENIAĆ
after_ps_metrics = get_current_metrics(spark_ui_address)

# Analiza wyników i wydajności *misji głównych*

## Część 1 - Spark Core (RDD)

In [46]:
# Wczytanie wyników z pliku pickle
word_counts = sc.pickleFile(rdd_result_dir)

# Wyświetlenie 50 pierwszych elementów
result_sample = word_counts.take(50)
for item in result_sample:
    print(item)

('club', 'Manchester City', 1281910000, 101385, 25, 39, ['ST', 'CF', 'RB', 'CAM', 'RM', 'GK', 'LB', 'RWB', 'CB', 'CM', 'LM', 'LW', 'CDM', 'RW'])
('club', 'FC Barcelona', 1131875000, 91571, 24, 49, ['RWB', 'CM', 'CB', 'GK', 'ST', 'CF', 'CDM', 'LB', 'RB', 'RW', 'LW', 'LM', 'RM', 'CAM'])
('club', 'Real Madrid', 1050650000, 118934, 24, 38, ['CM', 'CB', 'RW', 'LW', 'RB', 'CDM', 'CAM', 'RM', 'LB', 'GK', 'ST', 'CF'])
('league', 'Premier League', 11049435000, 21002, 24, 2295, ['RWB', 'GK', 'RW', 'RB', 'CDM', 'LM', 'LW', 'CM', 'CB', 'LWB', 'LB', 'ST', 'CF', 'RM', 'CAM'])
('league', 'Serie A', 7729900000, 19340, 26, 1544, ['GK', 'CDM', 'RW', 'LB', 'LWB', 'CAM', 'RM', 'RB', 'RWB', 'LW', 'LM', 'ST', 'CF', 'CB', 'CM'])
('league', 'La Liga', 7612290000, 21643, 25, 1143, ['RW', 'CAM', 'RM', 'LWB', 'ST', 'CF', 'CM', 'CB', 'LB', 'RWB', 'GK', 'RB', 'LW', 'LM', 'CDM'])
('nationality', 'Brazil', 7025990000, 13337, 27, 2440, ['LM', 'LW', 'ST', 'CF', 'GK', 'LB', 'RM', 'CAM', 'RB', 'CM', 'CB', 'RWB', 'RW', '

In [47]:
subtract_metrics(after_rdd_metrics, before_rdd_metrics)

{
  "numTasks": 3628,
  "numActiveTasks": 0,
  "numCompleteTasks": 1004,
  "numFailedTasks": 0,
  "numKilledTasks": 0,
  "numCompletedIndices": 1004,
  "executorDeserializeTime": 9001,
  "executorDeserializeCpuTime": 4358148294,
  "executorRunTime": 249222,
  "executorCpuTime": 16483027046,
  "resultSize": 2151644,
  "jvmGcTime": 1594,
  "resultSerializationTime": 1284,
  "memoryBytesSpilled": 0,
  "diskBytesSpilled": 0,
  "peakExecutionMemory": 0,
  "inputBytes": 34499267,
  "inputRecords": 56937,
  "outputBytes": 1746,
  "outputRecords": 9,
  "shuffleRemoteBlocksFetched": 2139,
  "shuffleLocalBlocksFetched": 2631,
  "shuffleFetchWaitTime": 305,
  "shuffleRemoteBytesRead": 14328026,
  "shuffleRemoteBytesReadToDisk": 0,
  "shuffleLocalBytesRead": 19269117,
  "shuffleReadBytes": 33597143,
  "shuffleReadRecords": 12076,
  "shuffleWriteBytes": 22903187,
  "shuffleWriteTime": 1041478711,
  "shuffleWriteRecords": 9972
}


## Część 2 - Spark SQL (DataFrame)

In [48]:
df = spark.table(df_result_table)

# Wyświetlenie 50 pierwszych rekordów
df.show(50)

+-----------+------------------+-------------+------------+-------+-------------+--------------------+
|   category|              name|sum_value_eur|avg_wage_eur|avg_age|count_players|    player_positions|
+-----------+------------------+-------------+------------+-------+-------------+--------------------+
|  club_name|       Real Madrid|    1.05065E9|    118934.0|   24.0|           38|[CDM, CAM, RM, ST...|
|  club_name|   Manchester City|    1.28191E9|    101385.0|   25.0|           39|[CDM, LB, GK, RB,...|
|  club_name|      FC Barcelona|   1.131875E9|     91571.0|   24.0|           49|[ST, GK, CM, CDM,...|
|league_name|           La Liga|    7.61229E9|     21643.0|   25.0|         1143|[CM, CAM, ST, LW,...|
|league_name|    Premier League| 1.1049435E10|     21002.0|   24.0|         2295|[CAM, LW, CM, CDM...|
|league_name|           Serie A|     7.7299E9|     19340.0|   26.0|         1544|[GK, CAM, CF, ST,...|
|nationality|             Egypt|     1.9557E8|     17681.0|   26.0|      

In [49]:
subtract_metrics(after_df_metrics, before_df_metrics)

{
  "numTasks": 324,
  "numActiveTasks": 0,
  "numCompleteTasks": 145,
  "numFailedTasks": 0,
  "numKilledTasks": 0,
  "numCompletedIndices": 145,
  "executorDeserializeTime": 7067,
  "executorDeserializeCpuTime": 3047873357,
  "executorRunTime": 119931,
  "executorCpuTime": 30342004680,
  "resultSize": 1155180,
  "jvmGcTime": 4050,
  "resultSerializationTime": 270,
  "memoryBytesSpilled": 0,
  "diskBytesSpilled": 0,
  "peakExecutionMemory": 11906516208,
  "inputBytes": 483036914,
  "inputRecords": 796436,
  "outputBytes": 7432,
  "outputRecords": 9,
  "shuffleRemoteBlocksFetched": 32,
  "shuffleLocalBlocksFetched": 53,
  "shuffleFetchWaitTime": 0,
  "shuffleRemoteBytesRead": 9225228,
  "shuffleRemoteBytesReadToDisk": 0,
  "shuffleLocalBytesRead": 13113428,
  "shuffleReadBytes": 22338656,
  "shuffleReadRecords": 843849,
  "shuffleWriteBytes": 22338372,
  "shuffleWriteTime": 215685499,
  "shuffleWriteRecords": 843846
}


## Część 3 - Pandas API on Spark

In [50]:
# Odczytaj zawartość pliku JSON przy użyciu Pandas API on Spark
read_df = ps.read_json(ps_result_file, lines=True)

# Przekształć dane do formatu słownika (listy słowników)
result_list = read_df.to_dict(orient='records')

# Sformatuj dane jako ładnie sformatowany JSON i wyświetl je
import json
formatted_result = json.dumps(result_list, indent=2, ensure_ascii=False)

# Wyświetl zawartość
print(formatted_result)

/usr/lib/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


[
  {
    "avg_age": 25.0,
    "avg_wage_eur": 21643.0,
    "category": "league_name",
    "count_players": 1143,
    "name": "La Liga",
    "player_positions": [
      "CAM",
      "CB",
      "CDM",
      "CF",
      "CM",
      "GK",
      "LB",
      "LM",
      "LW",
      "LWB",
      "RB",
      "RM",
      "RW",
      "RWB",
      "ST"
    ],
    "sum_value_eur": 7612290000.0
  },
  {
    "avg_age": 24.0,
    "avg_wage_eur": 21002.0,
    "category": "league_name",
    "count_players": 2295,
    "name": "Premier League",
    "player_positions": [
      "CAM",
      "CB",
      "CDM",
      "CF",
      "CM",
      "GK",
      "LB",
      "LM",
      "LW",
      "LWB",
      "RB",
      "RM",
      "RW",
      "RWB",
      "ST"
    ],
    "sum_value_eur": 11049435000.0
  },
  {
    "avg_age": 26.0,
    "avg_wage_eur": 19340.0,
    "category": "league_name",
    "count_players": 1544,
    "name": "Serie A",
    "player_positions": [
      "CAM",
      "CB",
      "CDM",
      "CF",

In [51]:
subtract_metrics(after_ps_metrics, before_ps_metrics)

{
  "numTasks": 477,
  "numActiveTasks": 0,
  "numCompleteTasks": 226,
  "numFailedTasks": 0,
  "numKilledTasks": 0,
  "numCompletedIndices": 226,
  "executorDeserializeTime": 6715,
  "executorDeserializeCpuTime": 2782501965,
  "executorRunTime": 204486,
  "executorCpuTime": 41719718988,
  "resultSize": 1538619,
  "jvmGcTime": 5825,
  "resultSerializationTime": 418,
  "memoryBytesSpilled": 0,
  "diskBytesSpilled": 0,
  "peakExecutionMemory": 17164003344,
  "inputBytes": 759650061,
  "inputRecords": 1551475,
  "outputBytes": 2011,
  "outputRecords": 9,
  "shuffleRemoteBlocksFetched": 45,
  "shuffleLocalBlocksFetched": 70,
  "shuffleFetchWaitTime": 0,
  "shuffleRemoteBytesRead": 8278599,
  "shuffleRemoteBytesReadToDisk": 0,
  "shuffleLocalBytesRead": 14366883,
  "shuffleReadBytes": 22645482,
  "shuffleReadRecords": 870460,
  "shuffleWriteBytes": 22644932,
  "shuffleWriteTime": 217734453,
  "shuffleWriteRecords": 870454
}
